In [3]:
import os

In [4]:
%pwd

'c:\\Users\\hp\\end-to-end-text-summarization\\research'

In [5]:
os.chdir("../")

In [6]:
%pwd

'c:\\Users\\hp\\end-to-end-text-summarization'

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_URL : str
    local_data_file : Path
    unzip_dir : Path


In [8]:
from textsummarizer.constants import CONFIG_FILE_PATH,PARAMS_FILE_PATH
from textsummarizer.utils.common import read_yaml, create_directories


In [9]:
class configurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.data_source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir)
        
        return data_ingestion_config
    

In [10]:
import os
import urllib.request as request
import zipfile
from textsummarizer.logging import logger
from textsummarizer.utils.common import get_size

In [11]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
                )
            logger.info(f"{filename} download! with follwing info: \n{headers}")
        else:
            logger.info(f"{get_size(Path(self.config.local_data_file))} already exists")


    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns home"""

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
    


In [13]:
try:
    config = configurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-10-07 09:27:13,485: INFO:common: yaml file:config\config.yaml loaded successfully]
[2024-10-07 09:27:13,488: INFO:common: yaml file:params.yaml loaded successfully]
[2024-10-07 09:27:13,490: INFO:common: directory created at: artifacts]


[2024-10-07 09:27:13,491: INFO:common: directory created at: artifacts/data_ingestion]
[2024-10-07 09:27:20,820: INFO:3681689355: artifacts/data_ingestion/data.zip download! with follwing info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 20A6:379E54:19441A3:1A83D17:6703A974
Accept-Ranges: bytes
Date: Mon, 07 Oct 2024 09:27:20 GMT
Via: 1.1 varnish
X-Served-By: cache-acc9628-ACC
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1728293238.864742,VS0,VE2248
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: e85320ddab664f46f8dd7d689b1d97d7b8ff5222
Expires: Mon